In [6]:
import numpy as np
import pandas as pd
import re

In [9]:
unlabeled_train = pd.read_csv('./sentiment_analysis_dataset.csv',encoding = 'utf-8',error_bad_lines=False)
unlabeled_train.columns = ['class','text']
unlabeled_train.head()

b'Skipping line 5944: expected 2 fields, saw 8\n'


,class,text
0,neg,a sentimental mess that never rings true .
1,neg,"while the performances are often engaging , th..."
2,neg,"interesting , but not compelling ."
3,neg,on a cutting room floor somewhere lies . . . f...
4,neg,while the ensemble player who gained notice in...


In [25]:
X = unlabeled_train['text']
Y = unlabeled_train['class']

In [26]:
#将数据划分为训练集和测试集   留出法（hold-out）
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.30, random_state=42)

In [4]:
import gensim
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/PY3/english.pickle')

def review_to_wordlist( review, remove_stopwords=False ):
    
    review_text = re.sub("[^a-zA-Z]"," ", review)

    words = review_text.lower().split()

    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return(words)

def review_to_sentences( review, remove_stopwords=False ):

    sentences = []
    for raw_sentence in review:
        if len(raw_sentence) > 0:
            # 获取句子中的词列表
            sentences.append( review_to_wordlist( raw_sentence, remove_stopwords ))
    return sentences

[nltk_data] Downloading package punkt to /home/uptao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
sentences = []
for i, review in enumerate(X_train):
    sentences += review_to_sentences(review)

In [28]:
for review in X_train:
    sentences += review_to_sentences(review)

In [29]:
len(sentences)

1708450

In [30]:
#构建word2vec模型
import time
from gensim.models import Word2Vec
# 模型参数
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [31]:
%%time
# 训练模型
print("训练模型中...")
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

训练模型中...
CPU times: user 14 s, sys: 97.6 ms, total: 14.1 s
Wall time: 12.2 s


In [33]:
print( '保存模型...')
model.init_sims(replace=True)
model_name = "300features_40minwords_10context"
model.save(model_name)

保存模型...


In [53]:
def makeFeatureVec(words, model, num_features):
    '''
    对段落中的所有词向量进行取平均操作
    '''
    featureVec = np.zeros((num_features), dtype="float32")
    nwords = 0.
   
    # Index2word包含了词表中的所有词，为了检索速度，保存到set中
    index2word_set = set(model.wv.vocab)
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])

    # 取平均
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    '''
    给定一个文本列表，每个文本由一个词列表组成，返回每个文本的词向量平均值
    '''
    counter = 0.
    j = 0

    reviewFeatureVecs = np.zeros((len(reviews), num_features), dtype="float32")

    for review in reviews:
        if counter % 5000. == 0.:
               print("Review %d of %d" % (counter, len(reviews)))

        reviewFeatureVecs[j] = makeFeatureVec(review, model, num_features)

        counter = counter + 1.
        j = j+1
    return reviewFeatureVecs

In [54]:
%time trainDataVecs = getAvgFeatureVecs( X_train, model, num_features )

Review 0 of 7448


/home/uptao/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 5000 of 7448
CPU times: user 4.51 s, sys: 28.5 ms, total: 4.54 s
Wall time: 4.47 s


In [55]:
%time testDataVecs = getAvgFeatureVecs(X_test, model, num_features)

Review 0 of 3193


/home/uptao/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


CPU times: user 1.89 s, sys: 12.2 ms, total: 1.9 s
Wall time: 1.88 s


In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

forest = RandomForestClassifier( n_estimators = 100, n_jobs=2)

print("Fitting a random forest to labeled training data...")
%time forest = forest.fit( trainDataVecs, Y_train )
print ("随机森林分类器10折交叉验证得分: ", np.mean(cross_val_score(forest, trainDataVecs, Y_train, cv=10, scoring='roc_auc')))

# 测试集
result = forest.predict( testDataVecs )

#output = pd.DataFrame( data={"id":X_test["text"], "sentiment":result} )
#output.to_csv( "rf_word2vec.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
CPU times: user 13.3 s, sys: 7.37 ms, total: 13.4 s
Wall time: 6.78 s
随机森林分类器10折交叉验证得分:  0.587838143167249
